### imports

In [1]:
import pandas as pd
import numpy as np
import joblib
from xgboost import XGBRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

### Load Data & Define Columns

In [2]:
# Load Data
df = pd.read_csv('final_coffee_no_area_year.csv')

# Define Columns
feature_cols = [
    'Annual precipitation',
    'Temperature Change on Land',
    'Altitude',
    'Processing Method_Natural / Dry',
    'Processing Method_Other',
    'Processing Method_Pulped natural / honey',
    'Processing Method_Semi-washed / Semi-pulped',
    'Processing Method_Washed / Wet',
    'Processing Method_Anaerobico 1000h',
    'Processing Method_intrinsic cherry',
    'Processing Method_Double Anaerobic Washed',
    'Processing Method_Fermented in barrels with figs 4days',
    'Processing Method_Double Carbonic Maceration / Natural'
]

quality_cols = [
    'Aroma', 'Flavor', 'Aftertaste', 'Acidity', 'Body', 'Balance',
    'Uniformity', 'Clean Cup', 'Sweetness', 'Cupper Points'
]

target_moisture = 'Moisture'
target_yield = 'Total Yield'

### Load Trained RF Models

In [3]:
rf_quality = joblib.load('rf_quality_model.pkl')   # MultiOutputRegressor
rf_moisture = joblib.load('rf_moisture_model.pkl') # Regressor
rf_yield = joblib.load('rf_yield_model.pkl')       # Regressor

### Prepare Data

In [4]:
X = df[feature_cols]
y_quality = df[quality_cols]
y_moisture = df[[target_moisture]]
y_yield = df[[target_yield]]

# Consistent train-test split
train_idx, test_idx = train_test_split(df.index, test_size=0.2, random_state=42)
X_train = X.loc[train_idx]
X_test = X.loc[test_idx]

y_quality_train = y_quality.loc[train_idx]
y_quality_test = y_quality.loc[test_idx]

y_moisture_train = y_moisture.loc[train_idx]
y_moisture_test = y_moisture.loc[test_idx]

y_yield_train = y_yield.loc[train_idx]
y_yield_test = y_yield.loc[test_idx]

### QUALITY: MultiOutput XGBoost stacked on RF predictions

In [5]:
rf_quality_train_preds = rf_quality.predict(X_train)
rf_quality_test_preds = rf_quality.predict(X_test)

xgb_quality_base = XGBRegressor(n_estimators=100, max_depth=4, learning_rate=0.1, random_state=42)
xgb_quality = MultiOutputRegressor(xgb_quality_base)
xgb_quality.fit(rf_quality_train_preds, y_quality_train)

y_quality_pred = xgb_quality.predict(rf_quality_test_preds)
rmse_quality = np.sqrt(mean_squared_error(y_quality_test, y_quality_pred))
print(f"✅ Quality (10 metrics) RMSE (stacked on RF): {rmse_quality:.4f}")

✅ Quality (10 metrics) RMSE (stacked on RF): 0.2662


### MOISTURE: XGBoost stacked on RF prediction + features

In [6]:
rf_moisture_train_preds = rf_moisture.predict(X_train)
rf_moisture_test_preds = rf_moisture.predict(X_test)

X_train_moisture_stack = X_train.copy()
X_train_moisture_stack['rf_moisture'] = rf_moisture_train_preds

X_test_moisture_stack = X_test.copy()
X_test_moisture_stack['rf_moisture'] = rf_moisture_test_preds

xgb_moisture = XGBRegressor(n_estimators=100, random_state=42)
xgb_moisture.fit(X_train_moisture_stack, y_moisture_train)

y_moisture_pred = xgb_moisture.predict(X_test_moisture_stack)
rmse_moisture = np.sqrt(mean_squared_error(y_moisture_test, y_moisture_pred))
print(f"✅ Moisture RMSE: {rmse_moisture:.4f}")

✅ Moisture RMSE: 0.0252


### YIELD: XGBoost stacked on RF prediction + features

In [7]:
rf_yield_train_preds = rf_yield.predict(X_train)
rf_yield_test_preds = rf_yield.predict(X_test)

X_train_yield_stack = X_train.copy()
X_train_yield_stack['rf_yield'] = rf_yield_train_preds

X_test_yield_stack = X_test.copy()
X_test_yield_stack['rf_yield'] = rf_yield_test_preds

xgb_yield = XGBRegressor(n_estimators=100, random_state=42)
xgb_yield.fit(X_train_yield_stack, y_yield_train)

y_yield_pred = xgb_yield.predict(X_test_yield_stack)
rmse_yield = np.sqrt(mean_squared_error(y_yield_test, y_yield_pred))
print(f"✅ Yield RMSE: {rmse_yield:.4f}")

✅ Yield RMSE: 17791.4245


### Save Meta-Models

In [9]:
from sklearn.metrics import r2_score

# Calculate R² for Quality (stacked model)
r2_quality = r2_score(y_quality_test, y_quality_pred)
print(f"Quality (10 metrics) R² (stacked on RF): {r2_quality:.4f}")

# Calculate R² for Moisture (stacked model)
r2_moisture = r2_score(y_moisture_test, y_moisture_pred)
print(f"Moisture R²: {r2_moisture:.4f}")

# Calculate R² for Yield (stacked model)
r2_yield = r2_score(y_yield_test, y_yield_pred)
print(f"Yield R²: {r2_yield:.4f}")

# Create a summary DataFrame of the stacked model evaluation metrics
stacked_eval_summary = pd.DataFrame({
    'Model': ['Quality (Stacked)', 'Moisture (Stacked)', 'Yield (Stacked)'],
    'RMSE': [rmse_quality, rmse_moisture, rmse_yield],
    'R²': [r2_quality, r2_moisture, r2_yield]
})

print("\nStacked Models Evaluation Summary:")
print(stacked_eval_summary)

Quality (10 metrics) R² (stacked on RF): 0.7488
Moisture R²: 0.6102
Yield R²: 0.8902

Stacked Models Evaluation Summary:
                Model          RMSE        R²
0   Quality (Stacked)      0.266218  0.748803
1  Moisture (Stacked)      0.025163  0.610191
2     Yield (Stacked)  17791.424451  0.890166


In [10]:
from sklearn.metrics import f1_score
import numpy as np

# Function to convert regression values to binary classes using a threshold
def binarize_predictions(y_true, y_pred, threshold_percentile=50):
    threshold = np.percentile(y_true, threshold_percentile)
    y_true_binary = (y_true > threshold).astype(int)
    y_pred_binary = (y_pred > threshold).astype(int)
    return y_true_binary, y_pred_binary

# For quality, we need to average across all quality metrics (for the stacked model)
y_quality_test_mean = np.mean(y_quality_test, axis=1)
y_quality_pred_mean = np.mean(y_quality_pred, axis=1)

y_quality_true_bin, y_quality_pred_bin = binarize_predictions(y_quality_test_mean, y_quality_pred_mean)
f1_quality_stacked = f1_score(y_quality_true_bin, y_quality_pred_bin)

# For moisture (stacked model)
y_moisture_true_bin, y_moisture_pred_bin = binarize_predictions(y_moisture_test.values, y_moisture_pred)
f1_moisture_stacked = f1_score(y_moisture_true_bin, y_moisture_pred_bin)

# For yield (stacked model)
y_yield_true_bin, y_yield_pred_bin = binarize_predictions(y_yield_test.values, y_yield_pred)
f1_yield_stacked = f1_score(y_yield_true_bin, y_yield_pred_bin)

print(f"Quality Group F1 Score (Stacked, binary): {f1_quality_stacked:.4f}")
print(f"Moisture F1 Score (Stacked, binary): {f1_moisture_stacked:.4f}")
print(f"Total Yield F1 Score (Stacked, binary): {f1_yield_stacked:.4f}")

Quality Group F1 Score (Stacked, binary): 0.8253
Moisture F1 Score (Stacked, binary): 0.6741
Total Yield F1 Score (Stacked, binary): 0.9812


In [8]:
joblib.dump(xgb_quality, 'xgb_meta_quality_multioutput.pkl')
joblib.dump(xgb_moisture, 'xgb_meta_moisture.pkl')
joblib.dump(xgb_yield, 'xgb_meta_yield.pkl')

['xgb_meta_yield.pkl']